# LangChain Expression Language (LCEL)

## What is ?

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. 

## Pro

There are several benefits to writing our chains using LCEL as opposed to writing normal code

1. **Async, Batch, and Streaming Suppor**: you can run your chains automatically full sync, async, batch and streaming without rewriting them.
2. **Fallbacks:** The non-determinism behaivor of LLMs makes it important to be able to handle erros easily. With LCEL we can attach fallbacks to any chain. 
3. **Parallelism:** With LCEL syntax you can run any component in parallel automatically.

## 1. Installation

This will install the bare minimum requirements of LangChain. if you are gonna request to opeanAI models, it must be to install openai library

In [ ]:
#!pip install langchain

## 2. Environment setup

Depend on your LLM provider, it's necessary to install their Python package, E.g: we're gonna use openAI models:

In [ ]:
#!pip install openai

let's create a .env file with credentials

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

or set within notebook.

In [ ]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "<>"
os.environ["OPENAI_API_BASE"] = "..."
os.environ["OPENAI_API_KEY"] = "..."

Note: In this example we're are using azureOpenAI services

## Chains

Chains are sequences of instructions the LangChain framework executes to perform a task.

To make it as easy as possible to create custom chains, Langchain have implemented a "Runnable" protocol.


## "Runnable" protocol

This is a standard interface, which makes it easy to define custom chains as well as invoke them in a standard way. 

The standard interface includes (ways to run a chain):

## Ways to run a chain sync

- **stream:** stream back chunks of the response
- **invoke:** call the chain on an input
- **batch:** call the chain on a list of inputs


## Ways to run a chain async

These also have corresponding async methods:

- **astream:** stream back chunks of the response async
- **ainvoke:** call the chain on an input async
- **abatch:** call the chain on a list of inputs async
- **astream_log:** stream back intermediate steps as they happen, in addition to the final response

The **input type** varies by component:

| Component | Input Type |
| --- | --- |
|Prompt|Dictionary|
|Retriever|Single string|
|LLM, ChatModel| Single string, list of chat messages or a PromptValue|
|Tool|Single string, or dictionary, depending on the tool|
|OutputParser|The output of an LLM or ChatModel|

The **output type** also varies by component:

| Component | Output Type |
| --- | --- |
| LLM | String |
| ChatModel | ChatMessage |
| Prompt | PromptValue |
| Retriever | List of documents |
| Tool | Depends on the tool |
| OutputParser | Depends on the parser |

All **runnables** expose **input** and **output** schemas to inspect the inputs and outputs:
    
- **input_schema:** an input Pydantic model auto-generated from the structure of the Runnable
- **output_schema:** an output Pydantic model auto-generated from the structure of the Runnable


## Chain => (PromptTemplate + ChatModel) 

Let's create our frist chain using prompt + ChatModel

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI

In [ ]:
model = AzureChatOpenAI(temperature=0.0, deployment_name=  "GPT35-16k", model_name="gpt-35-turbo-16k")
prompt = ChatPromptTemplate.from_template("just tell me the name of most typical course from:{country} and its ingredients")

chain = INPUT | MODEL | OUTPUT

In [ ]:
chain = prompt | model

## Input Schema

In [ ]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.input_schema.schema()

In [ ]:
prompt.input_schema.schema()

The chain and prompt input are the same, it's logic 

In [ ]:
chain.input_schema.schema() == prompt.input_schema.schema()

In [ ]:
model.input_schema.schema()

## Output Schema

In [ ]:
chain.output_schema.schema()

In [ ]:
model.output_schema.schema()

In [ ]:
chain.output_schema.schema() == model.output_schema.schema()

# [ Sync ] Ways to run a chain 

## Stream

In [ ]:
for s in chain.stream({"country": "Colombia"}):
    print(s.content, end="", flush=True)

## Invoke

In [ ]:
chain.invoke({"country" :"Colombia"})

## Batch

In [ ]:
chain.batch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}])

We can set the number of concurrent requests by using the max_concurrency parameter

In [ ]:
chain.batch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}],
           config={"max_concurrency": 2})

# [ Async ] Ways to run a chain 

## Async Stream

In [ ]:
async for s in chain.astream({"country": "Colombia"}):
    print(s.content, end="", flush=True)

## Async Invoke

In [ ]:
await chain.ainvoke({"country":"Argentina"})

## Async Batch

In [ ]:
await chain.abatch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}])

# Parallelism

Using a RunnableParallel (often written as a dictionary) it executes each element in parallel.

In [ ]:
from langchain.schema.runnable import RunnableParallel

In [ ]:
course_name_prompt = ChatPromptTemplate.from_template("just tell me the name of most typical course from:{country}")
course_name_chain =  course_name_prompt| model

location_prompt = ChatPromptTemplate.from_template("tell me the location of :{country}")
location_chain = location_prompt | model

In [ ]:
%%timeit -r 1
print(course_name_chain.invoke({"country":"Colombia"}))

In [ ]:
%%timeit -r 1
print(location_chain.invoke({"country":"Colombia"}))

Now, Run it parallel

In [ ]:
parallel_chain = RunnableParallel(food=course_name_chain, location=location_chain)

In [ ]:
%%timeit -r 1
print(parallel_chain.invoke({"country":"Colombia"}))

## Parallelism on batches

In [ ]:
%%timeit -r 1
print(course_name_chain.batch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}]))

In [ ]:
%%timeit -r 1
print(location_chain.batch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}]))

In [ ]:
%%timeit -r 1
print(parallel_chain.batch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}]))

In [ ]:
summary_chain  = prompt_summary | chat_model | parserOutput

In [ ]:
key_point_chain = prompt_key_point | chat_model 

In [ ]:
chain_summ_key = summary_chain | key_point_chain